# HepMC3 Interface: IO and visualization

chromo uses the pyhepmc library for event IO and visualization. Visualization of the event is useful to understand the particle history. Not all generators produce a history. Here we generate an event with Pythia6 which has full history.

For the visualization to work, the optional package graphviz is required.

In [ ]:
import chromo.models as im
from chromo.kinematics import CenterOfMass
from chromo.constants import GeV
from chromo.util import pdg2name
from particle import literals as lp, Particle

In [ ]:
evt_kin = CenterOfMass(15 * GeV, "p", "p")

model = im.Pythia6(evt_kin, seed=1)

# generate one event
for event in model(1):
    pass

To get a look at the event, we can `print` it, but the output is a bit confusing. chromo gives access to the Fortran HepEvt record, which is a very compact and efficient format, but it is not easy to see what is going on.

In [ ]:
print(event)

To get a better visualization, we can convert the event into a directed graph. The HepMC3 library provides such a graph structure. We convert the chromo event to HepMC3 format.

In [ ]:
hev = event.to_hepmc3()

The HepMC3 event consists of particles and vertices. You can traverse the graph up and down to follow all ancestors of a final state particle.

In [ ]:
# get one final state particle
for p in hev.particles:
    if p.status == 1: # final state
        break

# print particle and its parents
while True:
    print(f"{pdg2name(p.pid)} E={p.momentum.e / GeV:.3g} status={p.status}")
    if not p.production_vertex.particles_in:
        break
    p = p.production_vertex.particles_in[0]
    

If you have `graphviz` installed, the HepMC3 event will be visualized directly in the notebook.

In [ ]:
hev

Try hovering the mouse over the lines and nodes to get extra tooltip information about the particles and vertices.

If you have all these extra packages installed, you can also display the original chromo event in this way (it is converted automatically then to generate this visualization).

In [ ]:
event

This scalable SVG image is generated with the function `pyhepmc.view.to_dot`, which produces a `DiGraph` object. It is possible to manipulate this object, for example, to change the size of the image. For details, see the `graphviz` docs.

In [ ]:
from pyhepmc.view import to_dot

In [ ]:
d = to_dot(hev)
d.graph_attr["size"] = "10,10"
d

Also important is the ability to save events and to pass them to other HEP software. The HepMC format is the common language in particle physics, which all tools understand. `pyhepmc` offers a Pythonic way to read/write events with an `open` function.

chromo events can be written to disk directly without manual conversion to HepMC3 format, because `pyhepmc` respects the following protocol. If a foreign event object has a method called `to_hepmc3`, it is called to convert the event automatically before writing.

In [ ]:
import pyhepmc

with pyhepmc.open("pythia6.dat", "w") as f:
    f.write(event)  # both event and hev work here

The format is human-readable ASCII.

In [ ]:
with open("pythia6.dat") as f:
    print(f.read())

We can also handle incomplete records, like the one from SIBYLL-2.1.

In [ ]:
evt_kin = CenterOfMass(20 * GeV, "p", "p")

model = im.Sibyll21(evt_kin, seed=1)

# generate one event
for event2 in model(1):
    pass

Sibyll-2.1 does not include the beam particles in the output, so there is no common production vertex for all particles.

In [ ]:
hev2 = event2.to_hepmc3()
hev2

The serialized output is more compact. For SIBYLL-2.1, the vertex positions are all zero, and the HepMC3 format then optimizes the vertex entries away.

In [ ]:
with pyhepmc.open("sibyll21.dat", "w") as f:
    f.write(event2)
    
with open("sibyll21.dat") as f:
    print(f.read())